<font size=6 bold=true>Dec 29 -- Part B
<br><br>Content Acquisition</font>

<font size=5 color='Olive'>For the first 5 lessons, DS will be combined with Python due to insufficiencies of Concepts</font>

In [1]:
import re
import sys
import csv
import pandas as pd
import numpy as np
import os
from os import path
from datetime import datetime
from urllib.parse import urlparse, urljoin
import requests
import ntpath
from bs4 import BeautifulSoup, Comment
import time
from pprint import pprint 
import shutil

<h1>Object Oriented Programming: Define Classes</h1>

In [2]:
class Describe_College(object):    
    def __init__(self,col_name, avg_gpa, avg_act, avg_sat, addr): 
        self.col_name=col_name
        self.avg_gpa=avg_gpa
        self.avg_act=avg_act
        self.avg_sat=avg_sat
        self.addr=addr 
    def basic_info(self):
        print("College: {} avg gpa requirement is {}".format(self.col_name,self.avg_gpa))

    def get_avggpa(self):
        return self.avg_gpa

c=Describe_College('Stanford', 3.0, 33.0, 1500, 'CA')
c.basic_info()
c.get_avggpa()

College: Stanford avg gpa requirement is 3.0


3.0

In [59]:
'''
    c=CollegeCrawl('University of Washington', 'https://www.washington.edu')
    c.GetAllUrls()
'''
class CollegeCrawl():
    Gap_Insecond=1
    Max_Pages=15      
    """
        collegename: name
        rooturl: https://www.university.edu. Scheme and netloc need to be complete
        prioritykeywords: ['apply','adimission'...] etc. if None then everth page 
        respectrobottxt: True
    """
    def __init__(self,_collegename, _rooturl, 
                 _prioritykeywords=['apply', 'admission', 'application', 'deadline'], 
                 _url_file=None,                  
                 _save_to_folder=None,
                 _existingurlfile=None, #csv files that were visited in the past
                 _respectrobottxt=True, 
                _headers={'User-Agent':'Mozilla/5.0'} ):
        self.college=_collegename
         
        if urlparse(_rooturl).scheme=="":
            print('URL needs to have scheme. Please try again.')
            raise Exception('URL needs to have scheme')            
            return
        
        self.rootUrl=_rooturl
        self.base_domain=urlparse(self.rootUrl).netloc
        self.scheme=urlparse(self.rootUrl).scheme
        self.priorityKeywords=_prioritykeywords
        self.respectRobottext=_respectrobottxt
        if _save_to_folder==None or path.isdir(_save_to_folder)==False:
            self.save_to_folder=os.getcwd()
        else:
            self.save_to_folder=_save_to_folder
            
        #to make it less 
        if _existingurlfile==None or path.exists(_existingurlfile)==False:            
            self.existingurlfile=path.join(self.save_to_folder,re.sub("\s+", "_", self.college.strip()+'.csv'))
        else:
            self.existingurlfile=_existingurlfile
        self.allurls={}
        self.headers=_headers
        self.files=[]
            
    '''simple description'''        
    def __str__(self):
        return '{}. Starting URL: {}'.format(self.college, self.rootUrl)
    '''
        load _existingurlfile: two columns -- Url and status_code
        minimum assumptions: first two columns are url and status_code
    '''
    def Load_DiscoveredUrls(self, delimiter=',', hasHeader=False, header_names=['url', 'status_code']):   
        if self.existingurlfile==None:
            return {}
        else:
            if path.exists(self.existingurlfile) and re.sub('\s+', '_', self.college) in ntpath.basename(self.existingurlfile):
                df_urls=pd.read_csv(self.existingurlfile,  delimiter=delimiter).iloc[:, 0:2]   
                header_row=list(df_urls.columns) 
            
                if re.match('^http', header_row[0]):
                    df_urls.columns=header_names
                    df_urls=pd.concat([df_urls, pd.DataFrame([header_row], columns=header_names)])                
                else: #event it is not, still assign the header_name
                    df_urls.columns=header_names 
                
                return dict(zip(df_urls['url'], df_urls['status_code'])) #format: url:status_code. i.e., url is the key
                #another options is: df_urls.to_dict('list') #format of {url:[url1, url2...], status_code:[0, 200...]}
            else:
                return {}        
            
    """
        get all urls starting from rootUrl 
        headers={'User-Agent':'Mozilla/5.0'}
        #Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.88 Safari/537.36
        #full list: https://www.whatismybrowser.com/guides/the-latest-user-agent/chrome
        response= requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text)  
        
        it will first load which ever has alread visited 

    """    
    def GetAllUrls(self, headers=None, links_only=True): 
        #load existing urls if any
        urls=self.Load_DiscoveredUrls() 
        
        if len(urls)==0:
            urls={self.rootUrl:0}  
            unvisited=[self.rootUrl]
        else:
            unvisited=[url for url, status_code in urls.items() if status_code==0]
            if not unvisited:
                unvisited=[self.rootUrl]
            
        if headers==None: 
            if self.headers:
                headers=self.headers
            else: 
                 headers={'User-Agent':'Mozilla/5.0'}  
                    
        #get base_domain    
        if self.base_domain==None:
            self.base_domain=urlparse(unvisited[0]).netloc
        if self.scheme==None:
            self.scheme=urlparse(unvisited[0]).scheme
     
        pages_visited =0
        try:  
            while unvisited:        
                pages_visited+=1
                url=unvisited.pop()  
                response=requests.get(url, headers=headers)     
                status_code=response.status_code
                urls[url]=status_code
                 
                if status_code==200:
                    soup=BeautifulSoup(response.text, 'html.parser') 
                    for link in soup.find_all('a'): 
                        if link.has_attr('href'):
                            url=link['href']       
                            url=re.sub(r'[\/_+!@#$?\\\s]+$', '', url)
                            parsed_uri = urlparse(url) 
                            absolute_url=''
           
                            if (parsed_uri.netloc=='') and (parsed_uri.scheme=='') and re.match(r'^\/.*\w$', parsed_uri.path) :
                                absolute_url=urljoin(self.scheme+'://'+self.base_domain,parsed_uri.path)
                            elif parsed_uri.netloc==self.base_domain and re.match('^http', parsed_uri.scheme):
                                absolute_url=url
                            elif parsed_uri.netloc==self.base_domain and parsed_uri.scheme=="" and re.match(r'^\/.*\w$', parsed_uri.path):
                                absolute_url=self.scheme+'://'+parsed_uri.netloc+parsed_uri.path
                            else:
                                continue    
                            if absolute_url!='' and absolute_url not in urls:   
                                 urls[absolute_url]=0
                  
                    self.SaveToCsv_FromResponse(url, response)  
              
                if pages_visited>=self.Max_Pages:
                    break 
             
                unvisited=[name for name, code in urls.items() if code==0]    
                #sorting rule: has keywords, short url, else
                unvisited=sorted(unvisited, key=lambda item: (sum([w in item for w in self.priorityKeywords])*10+10)/len(item))
                time.sleep(self.Gap_Insecond) #wait for few seconds. 
          
        except: 
            print('url "{}" went wrong'.format(url))  
            urls[url]=999
                #not to consider failed pages. status_code 400s may need manual handling of they are high priority pages
        finally: 
            self.allurls=urls
            #csv_columns = ['url', 'status_code']  
            #try:
                #with open(self.existingurlfile, 'w', newline='', encoding='utf-8') as csvfile:
                    #writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
                    #writer.writeheader()
                    #for data in urls:
                        #writer.writerow(data)
            #except IOError:
                #print("I/O error")
            self.Save_Summaries()
            
    """
        display summaries 
    """
    
    def Save_Summaries(self): 
        if self.allurls:
            df_urls=pd.DataFrame(list(c.allurls.items()), columns=['url', 'status_code'])
            print('Summary for college ', self.college)
            print('\n')
            print(df_urls.groupby('status_code').count().reset_index())
            #save file as well  
            try:
                with open(self.existingurlfile, 'w', newline='', encoding='utf-8') as csvfile:
                    writer = csv.writer(csvfile, delimiter=',', quoting=csv.QUOTE_MINIMAL)
                    writer.writerow(['url','status_code'])
                    for url, status_code in self.allurls.items():  
                        writer.writerow([url, status_code])  
            except IOError:
                print('IO Error in saving summaries')
                
        if self.files:
            print('\nThe following {} file(s) are generated. '.format(len(self.files)))
            pprint(self.files) 
    """
        read one page
    """
    def Read_Oneurl(self, url):  
        response=requests.get(url, self.headers)  
        if response.status_code==200: 
            return self.Get_Pagetext(response)
        else: 
            return [[None, None, None, None]]
    
    '''
        used for filter()
    '''
    def Tag_Visible(self, element):
        if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
            return False
        if isinstance(element, Comment):
            return False
        return True 

    '''
        get all text from page body
    '''
    def Get_Pagetext(self, response):
        soup = BeautifulSoup(response.text, 'html.parser') #conent
        texts = soup.findAll(text=True) 
        visible_texts = filter(self.Tag_Visible, texts)       

        return [ [t.parent.name,   
                 t.parent.previousSibling.name if t.parent.previousSibling!=None else None, 
                 t.nextSibling.name if t.nextSibling!=None else None,
                 re.sub(r'[\s+\t]',' ',t) ]  for t in visible_texts if len(t.strip())>2] 

    """
        Save One Page
        filename, if not None, should not be full name. use import ntpath ntpath.basename("a\b\c")
    """
    def SaveToCsv_FromUrl(self,url): # tab delimiter only 
        url=url.strip() 
        content=self.Read_Oneurl(url) #in format of (a,a,a,a)   
        self.SaveToCsv(url, content)
        #return SaveToCsv_FromResponse()
    
    '''
        save from resonse. called in the initial loop
    '''
    def SaveToCsv_FromResponse(self, url, response):
        content= self.Get_Pagetext(response)
        self.SaveToCsv(url, content) 
        
    '''
        save to csv file and append it to self.files
    '''    
    def SaveToCsv(self, url, content):
        filename=url.replace('.', '_dot_').replace('/', '_').replace(':', '_')+'_'+datetime.now().strftime("%m_%d_%Y_%H_%M_%S")+'.csv'
        fullname=path.join(self.save_to_folder, filename)
        try:
            with open(fullname, 'w', newline='', encoding='utf-8') as csvfile:
                writer = csv.writer(csvfile, delimiter='\t', quoting=csv.QUOTE_MINIMAL)
                writer.writerow(['url', 'parent', 'ps', 'ns', 'text'])
                for lll in content: 
                    lll.insert(0, url)
                    writer.writerow(lll)  
            self.files.append(fullname) 
        except IOError:
            print('failed to save file.')
            
    '''
        move file from one folder to another 
        pattern: regular expression
    '''
    def MoveFiles(self, destination_folder, source_folder=None,  pattern=None): 
        if source_folder==None:
            source_folder=self.save_to_folder
        if source_folder==None:
            print('Please specify folder with the files. ')
            return
        if destination_folder==None or path.isdir(destination_folder)==False:
            print('Need a valid destination folder')
            
        if pattern==None or pattern=="":
            pattern="csv$" 
        files = [f for f in os.listdir(self.save_to_folder) if re.match(pattern, f) and path.isfile(path.join(src, f))]
        for f in files:
            shutil.move(path.join(src, f), destination_folder)  
            

In [13]:
c=CollegeCrawl('washington', 'https://www.washington.edu/studentlife/')

In [1]:
uom=CollegeCrawl('michigan', 'https://umich.edu/')

NameError: name 'CollegeCrawl' is not defined

In [14]:
 c.GetAllUrls()

Summary for college  washington


   status_code  url
0            0  248
1          200   16

The following 16 file(s) are generated. 
['C:\\Coding\\utility_git\\Notebooks\\http___www_dot_washington_dot_edu_online_terms_12_29_2019_13_14_17.csv',
 'C:\\Coding\\utility_git\\Notebooks\\http___www_dot_washington_dot_edu_online_terms_12_29_2019_13_14_18.csv',
 'C:\\Coding\\utility_git\\Notebooks\\http___www_dot_washington_dot_edu_online_terms_12_29_2019_13_14_19.csv',
 'C:\\Coding\\utility_git\\Notebooks\\http___www_dot_washington_dot_edu_online_terms_12_29_2019_13_14_21.csv',
 'C:\\Coding\\utility_git\\Notebooks\\http___www_dot_washington_dot_edu_online_terms_12_29_2019_13_14_22.csv',
 'C:\\Coding\\utility_git\\Notebooks\\http___www_dot_washington_dot_edu_online_terms_12_29_2019_13_14_24.csv',
 'C:\\Coding\\utility_git\\Notebooks\\http___www_dot_washington_dot_edu_online_terms_12_29_2019_13_14_25.csv',
 'C:\\Coding\\utility_git\\Notebooks\\http___hr_dot_uw_dot_edu_workplace-posters_12_29_

In [69]:
def MoveFiles(  destination_folder=None, source_folder=None,  pattern=None): 
    if source_folder==None:
        source_folder='C:\\Coding\\utility_git\\Notebooks'
    if source_folder==None:
        print('Please specify folder with the files. ')
        return
    if destination_folder==None or path.isdir(destination_folder)==False:
        print('Need a valid destination folder')
    save_to_folder='c:/coding/hw'
    if pattern==None or pattern=="":
        pattern='.*.csv$' 
    return [f for f in os.listdir(source_folder) if re.match(pattern, f)]
    #files = [f for f in os.listdir(source_folder) if re.match(pattern, f) and path.isfile(path.source_folder(src, f))]
    return files, save_to_folder, [f for f in os.listdir(source_folder)]
    for f in files:
        shutil.move(path.join(src, f), destination_folder)  
            

In [70]:
a='http___hr_dot_uw_dot_edu_workplace-posters_12_29_2019_13_14_27.csv'
if re.match(r'.*.csv$', a):
    print('afd')

afd


In [71]:
MoveFiles()

Need a valid destination folder


['http___hr_dot_uw_dot_edu_workplace-posters_12_29_2019_13_14_27.csv',
 'http___www_dot_washington_dot_edu_online_terms_12_29_2019_13_13_23.csv',
 'http___www_dot_washington_dot_edu_online_terms_12_29_2019_13_13_24.csv',
 'http___www_dot_washington_dot_edu_online_terms_12_29_2019_13_14_17.csv',
 'http___www_dot_washington_dot_edu_online_terms_12_29_2019_13_14_18.csv',
 'http___www_dot_washington_dot_edu_online_terms_12_29_2019_13_14_19.csv',
 'http___www_dot_washington_dot_edu_online_terms_12_29_2019_13_14_21.csv',
 'http___www_dot_washington_dot_edu_online_terms_12_29_2019_13_14_22.csv',
 'http___www_dot_washington_dot_edu_online_terms_12_29_2019_13_14_24.csv',
 'http___www_dot_washington_dot_edu_online_terms_12_29_2019_13_14_25.csv',
 'http___www_dot_washington_dot_edu_online_terms_12_29_2019_13_14_28.csv',
 'http___www_dot_washington_dot_edu_online_terms_12_29_2019_13_14_30.csv',
 'http___www_dot_washington_dot_edu_online_terms_12_29_2019_13_14_31.csv',
 'http___www_dot_washington_d

In [76]:
a={3,4,5, 3, 4}
aset=set(a) 

In [77]:
a

{3, 4, 5}

In [78]:
aset

{3, 4, 5}

In [79]:
aset.pop()

3

In [80]:
alist=list(aset)

In [81]:
alist

[4, 5]

In [82]:
alist.pop()

5

In [83]:
help(alist.pop)

Help on built-in function pop:

pop(index=-1, /) method of builtins.list instance
    Remove and return item at index (default last).
    
    Raises IndexError if list is empty or index is out of range.



In [91]:
mycard={'mit':2, 'u':3}
ab={k: v for k, v in sorted(mycard.items(), key=lambda item: item[1])}

In [94]:
list(ab)

['mit', 'u']

In [89]:
help(sorted)

Help on built-in function sorted in module builtins:

sorted(iterable, /, *, key=None, reverse=False)
    Return a new list containing all items from the iterable in ascending order.
    
    A custom key function can be supplied to customize the sort order, and the
    reverse flag can be set to request the result in descending order.



In [ ]:
{key:value for key, value in sorted()}